In [1]:
import sqlite3
import os
import openpyxl
import pandas as pd
import calendar
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, MetaData
from sqlalchemy_schemadisplay import create_schema_graph
import matplotlib.image as mpimg

In [2]:
# Подключение к базе данных
conn = sqlite3.connect('bank_analytics.db')

In [3]:
# Создание курсора
cursor = conn.cursor()

In [4]:
# Выполните SQL-скрипт для создания таблицы Banks
create_banks_table_script = '''
    CREATE TABLE IF NOT EXISTS BANKS (
        "Bank_Id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "Наименование Банка" TEXT,
        "За период" DATE,
        "Кредиты на активы в %" REAL,
        "Депозиты на обязательства в %" REAL,
        "Кредиты на депозиты в %" REAL,
        "Доля операционных расходов в %" REAL,
        "Доля чистой прибыли в %" REAL,
        "Доля чистой процентной маржи за период в %" REAL,
        "CIR в %" REAL,
        "Доходность кредитного портфеля за период в %" REAL,
        "Стоимость депозиты за период в %" REAL,
        "ROA за период в %" REAL,
        "ROE за период в %" REAL,
        "Средняя заработная плата сотрудников за месяц в млн.сум" REAL,
        "Кредиты на депозиты ссуда к оплате в %" REAL,
        "ROA on EBITA в %" REAL,
        "Среднемесячная чистая прибыль на сотрудника" REAL,
        "ROE on EBITA в %" REAL,
        "COR за период в %" REAL,
        "Чистая прибыль в тысячах" REAL,
        "Чистая прибыль в тысячах по форме 2" REAL);
'''

In [5]:
cursor.execute(create_banks_table_script)

In [6]:
# Выполните SQL-скрипт для создания таблицы NBU
create_nbu_table_script = '''
    CREATE TABLE IF NOT EXISTS NBU 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_NBU Foreign key ("Bank_Id") references BANKS ("Bank_Id")); '''


In [7]:
cursor.execute(create_nbu_table_script)

In [8]:
# Выполните SQL-скрипт для создания таблицы ASAKA
create_asaka_table_script = '''
    CREATE TABLE IF NOT EXISTS ASAKA 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_ASAKA Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''


In [9]:
cursor.execute(create_asaka_table_script)

In [10]:
# Выполните SQL-скрипт для создания таблицы XALQ
create_xalq_table_script = '''
    CREATE TABLE IF NOT EXISTS XALQ 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_XALQ Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''


In [11]:
cursor.execute(create_xalq_table_script)

In [12]:
# Выполните SQL-скрипт для создания таблицы AsiaAliance
create_asiaaliance_table_script = '''
    CREATE TABLE IF NOT EXISTS ASIAALIANCE 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_ASIAALIANCE Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''


In [13]:
cursor.execute(create_asiaaliance_table_script)

In [14]:
# Выполните SQL-скрипт для создания таблицы Пойтахт
create_poytaxt_table_script = '''
    CREATE TABLE IF NOT EXISTS POYTAXT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_POYTAXT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''


In [15]:
cursor.execute(create_poytaxt_table_script)

In [16]:
# Выполните SQL-скрипт для создания таблицы SQB
create_sqb_table_script = '''
    CREATE TABLE IF NOT EXISTS SQB 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_SQB Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [17]:
cursor.execute(create_sqb_table_script)

In [18]:
# Выполните SQL-скрипт для создания таблицы ipotekabank
create_ipotekabank_table_script = '''
    CREATE TABLE IF NOT EXISTS IPOTEKABANK 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_IPOTEKABANK Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [19]:
cursor.execute(create_ipotekabank_table_script)

In [20]:
# Выполните SQL-скрипт для создания таблицы agrobank
create_agrobank_table_script = '''
    CREATE TABLE IF NOT EXISTS AGROBANK 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_AGROBANK Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [21]:
cursor.execute(create_agrobank_table_script)

In [22]:
# Выполните SQL-скрипт для создания таблицы rbb
create_rbb_table_script = '''
    CREATE TABLE IF NOT EXISTS RBB 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_RBB Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [23]:
cursor.execute(create_rbb_table_script)

In [24]:
# Выполните SQL-скрипт для создания таблицы turon
create_turon_table_script = '''
    CREATE TABLE IF NOT EXISTS TURON 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_TURON Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [25]:
cursor.execute(create_turon_table_script)

In [26]:
# Выполните SQL-скрипт для создания таблицы microcredit
create_microcredit_table_script = '''
    CREATE TABLE IF NOT EXISTS MICROCREDIT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_MICROCREDIT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [27]:
cursor.execute(create_microcredit_table_script)

In [28]:
# Выполните SQL-скрипт для создания таблицы aloqabank
create_aloqa_table_script = '''
    CREATE TABLE IF NOT EXISTS ALOQA 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_ALOQA Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [29]:
cursor.execute(create_aloqa_table_script)

In [30]:
# Выполните SQL-скрипт для создания таблицы ipakyuli
create_ipakyuli_table_script = '''
    CREATE TABLE IF NOT EXISTS IPAKYULI 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_IPAKYULI Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [31]:
cursor.execute(create_ipakyuli_table_script)

In [32]:
# Выполните SQL-скрипт для создания таблицы kapital
create_kapital_table_script = '''
    CREATE TABLE IF NOT EXISTS KAPITAL 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT,
            "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_KAPITAL Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [33]:
cursor.execute(create_kapital_table_script)

In [34]:
# Выполните SQL-скрипт для создания таблицы investfinance
create_investfinance_table_script = '''
    CREATE TABLE IF NOT EXISTS INVESTFINANCE 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_INVESTFINANCE Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [35]:
cursor.execute(create_investfinance_table_script)

In [36]:
# Выполните SQL-скрипт для создания таблицы universal
create_universal_table_script = '''
    CREATE TABLE IF NOT EXISTS UNIVERSAL 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_UNIVERSAL Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [37]:
cursor.execute(create_universal_table_script)

In [38]:
# Выполните SQL-скрипт для создания таблицы avobank
create_avo_table_script = '''
    CREATE TABLE IF NOT EXISTS AVO 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_AVO Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [39]:
cursor.execute(create_avo_table_script)

In [40]:
# Выполните SQL-скрипт для создания таблицы MADADINVEST
create_madadinvest_table_script = '''
    CREATE TABLE IF NOT EXISTS MADADINVEST 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_MADADINVEST Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [41]:
cursor.execute(create_madadinvest_table_script)

In [42]:
# Выполните SQL-скрипт для создания таблицы ofb
create_ofb_table_script = '''
    CREATE TABLE IF NOT EXISTS OFB 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_OFB Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [43]:
cursor.execute(create_ofb_table_script)

In [44]:
# Выполните SQL-скрипт для создания таблицы trast
create_trast_table_script = '''
    CREATE TABLE IF NOT EXISTS TRAST 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_TRAST Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [45]:
cursor.execute(create_trast_table_script)

In [46]:
# Выполните SQL-скрипт для создания таблицы Davrbank
create_davr_table_script = '''
    CREATE TABLE IF NOT EXISTS DAVR 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_DAVR Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [47]:
cursor.execute(create_davr_table_script)

In [48]:
# Выполните SQL-скрипт для создания таблицы Octobank
create_octobank_table_script = '''
    CREATE TABLE IF NOT EXISTS OCTOBANK 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_OCTOBANK Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [49]:
cursor.execute(create_octobank_table_script)

In [50]:
# Выполните SQL-скрипт для создания таблицы anorbank
create_anor_table_script = '''
    CREATE TABLE IF NOT EXISTS ANOR 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_ANOR Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [51]:
cursor.execute(create_anor_table_script)

In [52]:
# Выполните SQL-скрипт для создания таблицы uzumbank
create_uzum_table_script = '''
    CREATE TABLE IF NOT EXISTS UZUM 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_UZUM Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [53]:
cursor.execute(create_uzum_table_script)

In [54]:
# Выполните SQL-скрипт для создания таблицы apexbank
create_apex_table_script = '''
    CREATE TABLE IF NOT EXISTS APEX 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_APEX Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [55]:
cursor.execute(create_apex_table_script)

In [56]:
# Выполните SQL-скрипт для создания таблицы HAYOTbank
create_hayot_table_script = '''
    CREATE TABLE IF NOT EXISTS HAYOT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_HAYOT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [57]:
cursor.execute(create_hayot_table_script)

In [58]:
# Выполните SQL-скрипт для создания таблицы smartbank
create_smart_table_script = '''
    CREATE TABLE IF NOT EXISTS SMART 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_SMART Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [59]:
cursor.execute(create_smart_table_script)

In [60]:
# Выполните SQL-скрипт для создания таблицы yangibank
create_yangi_table_script = '''
    CREATE TABLE IF NOT EXISTS YANGI 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_YANGI Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [61]:
cursor.execute(create_yangi_table_script)

In [62]:
# Выполните SQL-скрипт для создания таблицы hamkorbank
create_hamkor_table_script = '''
    CREATE TABLE IF NOT EXISTS HAMKOR 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_HAMKOR Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [63]:
cursor.execute(create_hamkor_table_script)

In [64]:
# Выполните SQL-скрипт для создания таблицы QDBbank
create_qdb_table_script = '''
    CREATE TABLE IF NOT EXISTS QDB 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_QDB Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [65]:
cursor.execute(create_qdb_table_script)

In [66]:
# Выполните SQL-скрипт для создания таблицы garantbank
create_garant_table_script = '''
    CREATE TABLE IF NOT EXISTS GARANT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_GARANT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [67]:
cursor.execute(create_garant_table_script)

In [68]:
# Выполните SQL-скрипт для создания таблицы ziratbank
create_zirat_table_script = '''
    CREATE TABLE IF NOT EXISTS ZIRAT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_ZIRAT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [69]:
cursor.execute(create_zirat_table_script)

In [70]:
# Выполните SQL-скрипт для создания таблицы saderatbank
create_saderat_table_script = '''
    CREATE TABLE IF NOT EXISTS SADERAT 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_SADERAT Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [71]:
cursor.execute(create_saderat_table_script)

In [72]:
# Выполните SQL-скрипт для создания таблицы TBC
create_tbc_table_script = '''
    CREATE TABLE IF NOT EXISTS TBC 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_TBC Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [73]:
cursor.execute(create_tbc_table_script)

In [74]:
# Выполните SQL-скрипт для создания таблицы tengebank
create_tenge_table_script = '''
    CREATE TABLE IF NOT EXISTS TENGE 
            ("Bank_Id" INTEGER Primary key AUTOINCREMENT, "Наименование Банка" TEXT,
            "За период" DATE,
            "Активы Всего" REAL,
            "Активы в НацВалюте" REAL,
            "Активы в ИнВалюте эквсум" REAL,
            "К получению из других банков" REAL,
            "Ценные бумаги" REAL,
            "Инвестиции" REAL,
            "Кредиты всего" REAL,
            "Кредиты в НацВалюте" REAL,
            "Кредиты в ИнВалюте эквсум" REAL,
            "Созданные Резервы по Кредитам" REAL,
            "Кредиты до 365 дней" REAL,
            "Кредиты от 365 до 730 дней" REAL,
            "Кредиты свыше 2 лет" REAL,
            "Обязательства всего" REAL,
            "Обязательства в НацВалюте" REAL,
            "Обязательства в ИнВалюте эквсум" REAL,
            "К оплате ЦБ РУ" REAL,
            "К оплате другие банки" REAL,
            "Кредиты и лизинговые операции к оплате" REAL,
            "Выпущенные ценные бумаги" REAL,
            "Субординированные долговые обязательства" REAL,
            "Капитал" REAL,
            "Депозиты всего" REAL,
            "Депозиты физлиц ДВ" REAL,
            "Депозиты физлиц Срочный" REAL,
            "Депозиты физлиц Пластик" REAL,
            "Депозиты юрлиц ДВ" REAL,
            "Депозиты юрлиц Срочный" REAL,
            "Депозиты юрлиц другие" REAL,
            "Депозиты в НацВалюте" REAL,
            "Депозиты в ИнВалюте эквсум" REAL,
            "Депозиты до 365 дней" REAL,
            "Депозиты от 365 до 730 дней" REAL,
            "Депозиты свыше 2 лет" REAL,
            "Доходы всего" REAL,
            "Доходы в НацВалюте" REAL,
            "Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы всего" REAL,
            "Процентные Доходы в НацВалюте" REAL,
            "Процентные Доходы в ИнВалюте эквсум" REAL,
            "Процентные Доходы по счетам в другихбанках" REAL,
            "Процентные Доходы по кредитам" REAL,
            "Процентные Доходы другие" REAL,
            "Беспроцентные Доходы всего" REAL,
            "Беспроцентные Доходы в НацВалюте" REAL,
            "Беспроцентные Доходы в ИнВалюте эквсум" REAL,
            "Беспроцентные Доходы комиссионные" REAL,
            "Беспроцентные Доходы другие" REAL,
            "В том числе Возврат резервов" REAL,
            "В том числе Возврат списанных кредитов" REAL,
            "Расходы всего" REAL,
            "Расходы в НацВалюте" REAL,
            "Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы всего" REAL,
            "Процентные Расходы в НацВалюте" REAL,
            "Процентные Расходы в ИнВалюте эквсум" REAL,
            "Процентные Расходы по счетам в другихбанках" REAL,
            "Процентные Расходы по депозитам" REAL,
            "Процентные Расходы займам" REAL,
            "Беспроцентные Расходы всего" REAL,
            "Беспроцентные Расходы в НацВалюте" REAL,
            "Беспроцентные Расходы в ИнВалюте эквсум" REAL,
            "Операционные Расходы всего" REAL,
            "Операционные Расходы зарплата" REAL,
            "Чистая прибыль" REAL,
            "Чистая прибыль до уплаты налогов" REAL,
            "Созданные Резервы" REAL,
            "Оценка возможных убытков по кредитам и лизингу" REAL,
            "Оценка некредитных убытков" REAL,
            "Экономический норматив коэффициент достаточности капитала" REAL,
            "Экономический норматив коэффициент достаточности капитала 1 уровня" REAL,
            "Экономический норматив Левераж" REAL,
            "Экономический норматив коэффициент покрытия ликвидности" REAL,
            "Экономический норматив норма чистого стабильного финансирования" REAL,
            "Количество сотрудников" REAL,
            "Количество филиалов" REAL,
            "Количество банковских центров обслуживания" REAL,
            "ДБО пользователи Всего" REAL,
            "ДБО пользователи физлица" REAL,
            "ДБО пользователи юрлица" REAL,
            "Доходные активы" REAL,
            "Расходные обязательства" REAL,
            "Резервы по стандартным активам" REAL,
            "Возврат резеровов" REAL,
			Constraint FK_TENGE Foreign key ("Bank_Id") references Banks ("Bank_Id")); '''

In [75]:
cursor.execute(create_tenge_table_script)

In [76]:
# Примените изменения и закройте соединение
conn.commit()
conn.close()